# 🧬 Generación de Datos Sintéticos con SDV

**Versión:** v3_experimental  
**Rol:** Data Engineer (Synthetic Data)  
**Fecha:** 2026-01-08

---

## Objetivo

Generar datos sintéticos tabulares usando **GaussianCopulaSynthesizer** de SDV para aumentar el dataset de entrenamiento.

## Método

**GaussianCopula** es el método recomendado para datasets pequeños (<1000 filas) porque:
- No requiere muchos datos para aprender distribuciones
- Captura correlaciones entre variables
- Es estable y reproducible

## Restricciones

> ⚠️ **CERO LEAKAGE**: Solo se usa el archivo de Train. El Test está sellado.

---

In [ ]:
# ==============================================================================
# CONFIGURACIÓN Y DEPENDENCIAS
# ==============================================================================

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import json
from pathlib import Path
from datetime import datetime

# SDV
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.evaluation.single_table import evaluate_quality

# Configuración
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Paths
DATA_DIR = Path("../../v2/data/processed")
OUTPUT_DIR = Path(".")

# Cuántas filas sintéticas generar
N_SYNTHETIC = 1000

print("✅ Dependencias cargadas")
print(f"   SDV GaussianCopula")
print(f"   Random State: {RANDOM_STATE}")
print(f"   N Sintético: {N_SYNTHETIC}")

---
## 1. Carga de Datos de Entrenamiento

In [ ]:
# ==============================================================================
# CARGA DE DATOS (SOLO TRAIN - CERO LEAKAGE)
# ==============================================================================

train_df = pd.read_parquet(DATA_DIR / "train_final.parquet")

print(f"📊 Datos cargados (SOLO TRAIN):")
print(f"   Shape: {train_df.shape}")
print(f"   Columnas: {len(train_df.columns)}")

# Verificar columnas zero-variance a excluir
zero_var_cols = []
for col in train_df.columns:
    if train_df[col].nunique() <= 1:
        zero_var_cols.append(col)
        
print(f"\n⚠️ Columnas zero-variance detectadas: {zero_var_cols}")
print(f"   (Serán excluidas del sintetizador)")

# Crear dataframe limpio para síntesis
train_clean = train_df.drop(columns=zero_var_cols, errors='ignore')
print(f"\n📋 Dataset para síntesis: {train_clean.shape}")

---
## 2. Definición de Metadatos SDV

In [ ]:
# ==============================================================================
# DEFINICIÓN DE METADATOS
# ==============================================================================

print("📝 Definiendo metadatos para SDV...")

# Crear metadata automáticamente
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_clean)

# Mostrar tipos detectados
print(f"\n📋 Tipos detectados:")
for col, info in metadata.columns.items():
    sdtype = info.get('sdtype', 'unknown')
    print(f"   {col}: {sdtype}")
    
# Verificar metadata
metadata.validate()
print(f"\n✅ Metadatos validados correctamente")

In [ ]:
# ==============================================================================
# AJUSTE MANUAL DE METADATOS (si es necesario)
# ==============================================================================

# Asegurar que 'event' sea categórico/booleano
if 'event' in metadata.columns:
    metadata.update_column('event', sdtype='categorical')
    print("✏️ 'event' ajustado a categorical")

# Asegurar que 'duration' sea numérico
if 'duration' in metadata.columns:
    metadata.update_column('duration', sdtype='numerical')
    print("✏️ 'duration' ajustado a numerical")

# Asegurar que binarios tech_* sean categóricos
binary_cols = [c for c in train_clean.columns if c.startswith('tech_') or c == 'genero_m']
for col in binary_cols:
    if col in metadata.columns:
        metadata.update_column(col, sdtype='categorical')

print(f"✏️ {len(binary_cols)} columnas binarias ajustadas a categorical")

# Validar de nuevo
metadata.validate()
print("\n✅ Metadatos actualizados y validados")

---
## 3. Entrenamiento del Sintetizador

In [ ]:
# ==============================================================================
# CONFIGURAR Y ENTRENAR GAUSSIANCOPULASYNTHESIZER
# ==============================================================================

print("🔧 Configurando GaussianCopulaSynthesizer...")

synthesizer = GaussianCopulaSynthesizer(
    metadata,
    enforce_min_max_values=True,
    enforce_rounding=True,
    numerical_distributions={
        'duration': 'truncnorm',  # Truncated normal para evitar negativos
        'edad': 'truncnorm'
    },
    default_distribution='norm'
)

print("\n🏋️ Entrenando sintetizador...")
synthesizer.fit(train_clean)

print("\n✅ Sintetizador entrenado exitosamente")

---
## 4. Generación de Datos Sintéticos

In [ ]:
# ==============================================================================
# GENERAR DATOS SINTÉTICOS
# ==============================================================================

print(f"🧬 Generando {N_SYNTHETIC} filas sintéticas...")

synthetic_df = synthesizer.sample(num_rows=N_SYNTHETIC)

print(f"\n📊 Datos sintéticos generados:")
print(f"   Shape: {synthetic_df.shape}")
print(f"\n   Primeras filas:")
print(synthetic_df.head())

---
## 5. Post-Procesamiento

In [ ]:
# ==============================================================================
# POST-PROCESAMIENTO: ASEGURAR RESTRICCIONES DE DOMINIO
# ==============================================================================

print("🔧 Aplicando post-procesamiento...")

# 1. Duration debe ser > 0
min_duration = train_clean['duration'].min()
if 'duration' in synthetic_df.columns:
    before = (synthetic_df['duration'] <= 0).sum()
    synthetic_df['duration'] = synthetic_df['duration'].clip(lower=min_duration)
    print(f"   ✅ duration: {before} valores corregidos (clip a {min_duration:.2f})")

# 2. Event debe ser 0 o 1
if 'event' in synthetic_df.columns:
    synthetic_df['event'] = synthetic_df['event'].round().astype(int).clip(0, 1)
    print(f"   ✅ event: convertido a binario {synthetic_df['event'].unique()}")

# 3. Edad debe estar en rango razonable
if 'edad' in synthetic_df.columns:
    min_edad = train_clean['edad'].min()
    max_edad = train_clean['edad'].max()
    synthetic_df['edad'] = synthetic_df['edad'].clip(min_edad, max_edad).round().astype(int)
    print(f"   ✅ edad: clip a [{min_edad}, {max_edad}]")

# 4. Columnas binarias tech_* deben ser 0 o 1
binary_cols = [c for c in synthetic_df.columns if c.startswith('tech_') or c == 'genero_m']
for col in binary_cols:
    synthetic_df[col] = synthetic_df[col].round().astype(int).clip(0, 1)
print(f"   ✅ {len(binary_cols)} columnas binarias convertidas a 0/1")

# 5. Habilidades hab_* deben estar en [0, 1]
hab_cols = [c for c in synthetic_df.columns if c.startswith('hab_')]
for col in hab_cols:
    # Redondear a valores válidos: 0, 0.25, 0.5, 0.75, 1.0
    synthetic_df[col] = (synthetic_df[col] * 4).round() / 4
    synthetic_df[col] = synthetic_df[col].clip(0, 1)
print(f"   ✅ {len(hab_cols)} columnas hab_* normalizadas a [0, 0.25, 0.5, 0.75, 1.0]")

print("\n✅ Post-procesamiento completado")

---
## 6. Validación de Calidad

In [ ]:
# ==============================================================================
# VALIDACIÓN DE CALIDAD SDV
# ==============================================================================

print("📊 Evaluando calidad de los datos sintéticos...")

try:
    quality_report = evaluate_quality(
        real_data=train_clean,
        synthetic_data=synthetic_df,
        metadata=metadata
    )
    
    overall_score = quality_report.get_score()
    print(f"\n🏆 Score de Calidad General: {overall_score:.4f}")
    
except Exception as e:
    print(f"⚠️ No se pudo calcular el score de calidad: {e}")
    overall_score = None

In [ ]:
# ==============================================================================
# COMPARACIÓN ESTADÍSTICA
# ==============================================================================

print("📊 Comparación estadística Real vs Sintético:")
print("\n" + "="*70)

# Comparar estadísticas clave
comparison_cols = ['duration', 'event', 'edad']

for col in comparison_cols:
    if col in train_clean.columns and col in synthetic_df.columns:
        real_mean = train_clean[col].mean()
        synth_mean = synthetic_df[col].mean()
        real_std = train_clean[col].std()
        synth_std = synthetic_df[col].std()
        
        print(f"\n{col}:")
        print(f"   Real:      μ={real_mean:.3f}, σ={real_std:.3f}")
        print(f"   Sintético: μ={synth_mean:.3f}, σ={synth_std:.3f}")
        print(f"   Δ mean:    {abs(real_mean - synth_mean):.3f}")

# Comparar tasa de eventos
if 'event' in train_clean.columns:
    real_event_rate = train_clean['event'].mean()
    synth_event_rate = synthetic_df['event'].mean()
    print(f"\nEvent Rate:")
    print(f"   Real:      {real_event_rate:.1%}")
    print(f"   Sintético: {synth_event_rate:.1%}")
    print(f"   Δ:         {abs(real_event_rate - synth_event_rate):.1%}")

---
## 7. Guardar Resultados

In [ ]:
# ==============================================================================
# GUARDAR DATOS SINTÉTICOS Y MODELO
# ==============================================================================

print("💾 Guardando resultados...")

# 1. Guardar datos sintéticos
synthetic_df.to_parquet(OUTPUT_DIR / "synthetic_data_copula.parquet", index=False)
print(f"   ✅ synthetic_data_copula.parquet ({len(synthetic_df)} filas)")

# 2. Guardar modelo sintetizador
synthesizer.save(OUTPUT_DIR / "synthesizer_model.pkl")
print(f"   ✅ synthesizer_model.pkl")

# 3. Guardar metadatos
metadata.save_to_json(OUTPUT_DIR / "synthesizer_metadata.json")
print(f"   ✅ synthesizer_metadata.json")

# 4. Guardar reporte de generación
report = {
    "metadata": {
        "date": datetime.now().isoformat(),
        "method": "GaussianCopulaSynthesizer",
        "sdv_version": "1.32.0",
        "random_state": RANDOM_STATE
    },
    "input": {
        "n_real": len(train_clean),
        "n_features": len(train_clean.columns),
        "excluded_cols": zero_var_cols
    },
    "output": {
        "n_synthetic": len(synthetic_df),
        "quality_score": float(overall_score) if overall_score else None
    },
    "statistics_comparison": {
        "duration": {
            "real_mean": float(train_clean['duration'].mean()),
            "synth_mean": float(synthetic_df['duration'].mean()),
            "real_std": float(train_clean['duration'].std()),
            "synth_std": float(synthetic_df['duration'].std())
        },
        "event_rate": {
            "real": float(train_clean['event'].mean()),
            "synthetic": float(synthetic_df['event'].mean())
        }
    },
    "files_generated": [
        "synthetic_data_copula.parquet",
        "synthesizer_model.pkl",
        "synthesizer_metadata.json"
    ]
}

with open(OUTPUT_DIR / "generation_report.json", 'w') as f:
    json.dump(report, f, indent=2)
print(f"   ✅ generation_report.json")

print("\n" + "="*50)
print("🎉 GENERACIÓN SINTÉTICA COMPLETADA")
print("="*50)

---
## Resumen

### Archivos Generados

| Archivo | Descripción |
|---------|-------------|
| `synthetic_data_copula.parquet` | 1000 filas sintéticas |
| `synthesizer_model.pkl` | Modelo GaussianCopula entrenado |
| `synthesizer_metadata.json` | Metadatos SDV |
| `generation_report.json` | Reporte de generación |

### Siguiente Paso

**Prompt 5: Entrenamiento con Datos Aumentados** - Entrenar modelos con combinaciones de datos reales y sintéticos.

---